##**Gerrymandering-Environment**

    INITIAL STATE (provided externally via reset(options=...)):
        - 'district_map'
        - 'social_graph'
        - 'opinions'     

    ACTION:
        - new district assignment for each voter

    OBSERVATION (returned by reset/step):
        {
          'district_map'   : (num_voters,)
          'representatives': (num_districts,)  # voter indices; -1 if empty district
          'social_graph'   : (num_voters, num_voters)  # AUGMENTED: base social + rep->voter edges used for the step
          'opinions'       : (num_voters, 2)
          'opinion_graph'  : (num_voters, num_voters)  # similarity kernel derived from opinion distances
        }

    KEY LOGIC:
      - Representatives: for each district, pick the member that minimizes the sum of L2 distances to members in that district (discrete 1-median).
      - Opinion dynamics: DRF (assimilation/neutral/backfire) with weighted neighbor influence.
      - Reward: reduction in total distance to reference opinion c*


In [6]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.9 MB/s eta 0:00:00


In [7]:
import numpy as np
import torch
from torch_geometric.data import Data
import gymnasium as gym
from gymnasium import spaces

#This is the Frankenmandering Data class which includes:
*   social edge list: type: matrix and size : (v,v)
*   assignment edge list: type: matrix and size:(v,d)
*   number of orignin edge
*   geometric position
*   district label
*   edge attribute
*   representative
*   opinion



In [8]:
class FrankenData(Data):
  def __init__(self,so_edge, assignment, orig_edge_num, opinion,pos, reps,dist_label, edge_attr, **kwargs):
    super().__init__(**kwargs)

    self.orig_edge_num = orig_edge_num
    self.opinion = torch.tensor(opinion, dtype=torch.float32)
    self.pos = torch.tensor(pos, dtype=torch.float32)
    self.dist_label = torch.tensor(dist_label, dtype=torch.float32)

    #social graph
    self.so_edge = torch.tensor(so_edge, dtype=torch.long)
    self.edge_attr = torch.tensor(so_edge, dtype=torch.float32)

    # district graph
    self.dist_edge = torch.tensor(assignment, dtype=torch.long)

    self.reps = torch.tensor(reps, dtype=torch.long)

  def save(self,file_path):
      torch.save(self,file_path)
      print(f"FrankenData object saved to {file_path}")

  def load(self,file_path):
      data = torch.load(file_path)
      print(f"FrankenData object loaded from {file_path}")
      return data

In [11]:
class FrankenmanderingEnv(gym.Env):

    metadata = {"render_modes": ["human"]}

    def __init__(
        self,
        num_voters: int,
        num_districts: int,
        opinion_dim: int = 2,
        horizon: int = 10,
        seed: int | None = None,
        # target
        reference_opinion: np.ndarray | None = None,
        FrankenData = None
    ):
        super().__init__()
        self.num_voters = int(num_voters)
        self.num_districts = int(num_districts)
        self.opinion_dim = int(opinion_dim)
        self.horizon = int(horizon)
        self.rng = np.random.default_rng(seed)

        # target opinion c*
        if reference_opinion is None:
            self.c_star = np.zeros(self.opinion_dim, dtype=np.float32)
        else:
            self.c_star = np.asarray(reference_opinion, dtype=np.float32).reshape(self.num_voters,self.opinion_dim)

        # action spaces
        self.action_space = spaces.Box(low=0.0, high=1.0, shape=(self.num_voters, self.num_districts), dtype=np.float32)

        self.observation_space = spaces.Dict({
              "opinions": spaces.Box(low=-np.inf, high=np.inf, shape=(self.num_voters, self.opinion_dim), dtype=np.float32),
              "positions": spaces.Box(low=-np.inf, high=np.inf, shape=(self.num_voters, 2), dtype=np.float32),
              "dist_label": spaces.Box(low=0, high=self.num_districts-1, shape=(self.num_voters,), dtype=np.int32),
              "reps": spaces.Box(low=-1, high=self.num_voters-1, shape=(self.num_districts,), dtype=np.int32),
              "social_edge_index": spaces.Box(low=0, high=self.num_voters-1, shape=self._state.social_edge_index.shape, dtype=np.int32),
              "social_edge_attr": spaces.Box(low=0, high=np.inf, shape=self._state.social_edge_attr.shape, dtype=np.float32),
              "assignment": spaces.Box(low=0, high=self.num_voters-1, shape=self._state.district_edge_index.shape, dtype=np.int32),
        })

        # state
        self.t = 0
        self.FrankenData = FrankenData

    # options includes opinions, geometric position,  edge_index(social graph,assignment),assignment
    def reset(self, seed: int | None = None, options: dict | None = None):
        super().reset(seed=seed)
        self.t = 0

        if options is None:
            raise ValueError("reset(options=...) must provide 'opinions', 'pos', and either 'edge_index' or 'social_adj'.")

        # Opinions and positions
        x = np.asarray(options["opinions"], dtype=np.float32)
        pos = np.asarray(options["pos"], dtype=np.float32)

        # Social graph
        if "edge_index" in options:
            edge_index = np.asarray(options["edge_index"], dtype=np.int32)
            edge_attr = np.asarray(options.get("edge_attr", np.ones(edge_index.shape[1])), dtype=np.float32)
        else:
            adj = np.asarray(options["social_adj"], dtype=np.float32)
            edge_index, edge_attr = adj_to_coo(adj)

        # Assignment
        assignment = options.get("assignment", np.full((self.num_voters, self.num_districts),
                                                      1.0 / self.num_districts, dtype=np.float32))
        assignment = row_normalize(assignment)
        # assignment = ensure_non_empty(assignment)

        # District labels and representatives
        y = assignment.argmax(axis=1).astype(np.int32)
        reps = elect_representatives(y, x)

        # Augment district graph (edges for representative connections)
        aug_edge_index, aug_edge_attr = augment_with_reps(edge_index, edge_attr, reps, y)
        orig_edge_num = edge_index.shape[1]

        return self.FrankenData(
            orig_edge_num=orig_edge_num,
            so_edge= aug_edge_index,
            assignment = assignment,
            opinion=x,
            pos=pos,
            dist_label=y,
            reps=[r if r is not None else -1 for r in reps],
            aug_edge_attr=edge_attr
        ), {}

    def step(self, action: np.ndarray):
        if self.FrankenData is None:
            raise RuntimeError("Environment must be reset before calling step.")

        # Normalize action and compute new assignment
        assignment = row_normalize(np.asarray(action, dtype=np.float32))

        # assignment = ensure_non_empty(assignment)
        y = assignment.argmax(axis=1).astype(np.int32)

        # check the validity of district
        contiguous = check_contiguity(self.FrankenData.so_edge, y, self.num_districts)
        non_empty = all(np.any(y == d) for d in range(self.num_districts))

        if (not contiguous) or (not non_empty):
          terminated = False
          reward = -10
          return self.FrankenData, reward, terminated,False,{}

        # Elect new representatives
        reps = elect_representatives(y, self.FrankenData.opinion)

        # update social graph based on reps and new assignment
        aug_edge_index, aug_edge_attr = augment_with_reps(
            self.FrankenData.so_edge,
            self.FrankenData.edge_attr,
            reps,
            y
        )

        # Update oppinion
        x_new = opinion_update(aug_edge_index, aug_edge_attr, self.FrankenData.opinion)
        reward = self.reward(self.FrankenData.opinion, x_new, y, self.FrankenData.so_edge)

        self.t += 1
        terminated = self.t >= self.horizon

        return self.FrankenData(
              so_edge = aug_edge_index,
              orig_edge_num = aug_edge_index.shape[1],
              assignment=assignment,
              opinion= x_new,
              reps=[r if r is not None else -1 for r in reps],
              dist_label=y,
              aug_edge_attr=aug_edge_attr
            ), float(reward), terminated, False, {}


    def reward(self, oldX,newX, y, social_edge_index, a= 0.1, b =0.5):
        # check the population equity and compactness constraints
        pop_dev = population_equality(np.ones(y),y, self.num_districts)
        comp_score = compactness_score(social_edge_index, y)

        penalty =  -a * pop_dev - b * comp_score

        old_d=np.linalg.norm(oldX-self.c_star,axis=1).sum()
        new_d=np.linalg.norm(newX-self.c_star,axis=1).sum()

        reward = penalty + old_d-new_d
        return reward

    def render(self, mode="human"):
        mean = self._x.mean(axis=0)
        print(f"[t={self.t}] mean opinion ≈ {tuple(np.round(mean,3))}")

**Elect representatives**
###This function elects representative in each district using a spatial / metric voting rule, specifically the median voter rule

```
Args:
        y: (N,) array of district labels for each voter.
        X: (N, opinion_dim) array of voter opinions.
        num_districts: number of districts (K).

    Returns:
        reps: list of representative indices, one per district (None if district empty)
```



In [12]:
import numpy as np

def elect_representatives(y: np.ndarray, X: np.ndarray, num_districts: int):

    reps = [None] * num_districts
    N, m = X.shape

    # Precompute pairwise distances
    dif = X[:, None, :] - X[None, :, :]   # (N,N,m)
    dists = np.linalg.norm(dif, axis=2)   # (N,N) Euclidean distances

    for d in range(num_districts):
        members = np.where(y == d)[0]  # voters in district d
        if len(members) == 0:
            continue  # empty district

        # For each member, compute sum of distances to others in district
        sums = dists[np.ix_(members, members)].sum(axis=1)

        # Pick the member with the minimal total distance
        reps[d] = int(members[np.argmin(sums)])
    return reps

## **Augment with reps (G union H)**

###
Augment the social graph with representative edges (district graphs).
    
    Args:
        edge_index: (2, E) numpy array of existing social edges.
        edge_attr: (E,) numpy array of edge weights.
        reps: list of representative indices, one per district (None if empty).
        y: (N,) array of district assignments (each voter’s district label).
        rep_edge_weight: weight to assign to each representative edge (default=1.0).
    
    Returns:
        new_edge_index: (2, E+E_rep) with representative edges added.
        new_edge_attr:  (E+E_rep,) with weights for representative edges.

In [13]:
import numpy as np

def augment_with_reps(edge_index, edge_attr, reps, y, rep_edge_weight: float = 1.0):

    add_src, add_dst, add_w = [], [], []

    for d, r in enumerate(reps):
        if r is None:
            continue  # empty district
        members = np.where(y == d)[0]

        for v in members:
            if v == r:
                continue  # skip self-loop
            add_src.append(r)             # representative -> member
            add_dst.append(v)
            add_w.append(rep_edge_weight) # fixed influence weight

    if not add_src:  # no additional edges
        return edge_index, edge_attr

    rep_edges = np.vstack([add_src, add_dst])      # shape (2, E_rep)
    rep_weights = np.array(add_w, dtype=np.float32)

    new_edge_index = np.concatenate([edge_index, rep_edges], axis=1)
    new_edge_attr  = np.concatenate([edge_attr, rep_weights], axis=0)

    return new_edge_index, new_edge_attr


### **Opinion Update Function**
### At each step, voters "adjust" their position in opinion space based on the Euclidean distance to their neighbours, with the DRF


deciding whether they converge, diverge, or ignore.

Each neighbour "pulls" or "pushes" v depending on how similar they are.

The closer they are, the more v tends to assimilate.

If they’re a bit too far, v resists and moves in the opposite direction (backfire).

If they’re too far away, v ignores them.

The update is multi-dimensional, so if opinions are about multiple issues, the shift happens along the direction where the neighbour differs most.

Because normalization is used, v doesn’t "overreact" to having many neighbours.

    Apply opinion update based on DRF across graph edges.
    X: (N, opinion_dim)
    edge_index: shape (2, E)
    edge_attr: shape (E,)
    

In [14]:
def opinion_update(self, edge_index_aug, edge_attr, X: np.ndarray) -> np.ndarray:

    N, m = X.shape   # N voters, m-dimensional opinions
    newX = np.copy(X)

    updates = np.zeros_like(X)
    norm_factors = np.zeros((N, 1)) #Z

    for e in range(edge_index_aug.shape[1]):
        u, v = edge_index_aug[:, e]
        w = edge_attr[e]

        diff = X[u] - X[v]               # vector difference in opinion space

       # Euclidean distance
       # if diff is a 1-D array [x, y, z], then np.linalg.norm(diff, 2) calculates sqrt(x^2 + y^2 + z^2)
        dist = np.linalg.norm(diff, 2)

        if dist > 1e-8:  # avoid division by zero
            mu = drf(dist)
            direction = diff / dist      # unit vector (direction only)

            # influence: u influences v
            updates[v] += mu * w * direction
            norm_factors[v] += abs(w)

    # apply updates with normalization
    for v in range(N):
        if norm_factors[v] > 0:
            newX[v] = X[v] + updates[v] / norm_factors[v]

    return newX

**Discrepancy Response Function**
*   This function determines how a voter reacts when interacting with another voter based on the discrepancy (distance) between their opinions.
*   Tunable thresholds: epsilon values determine assimilation/backfire ranges. There are 5 levels in this range

### thresholds (tune these as hyperparameters)
*   eps_indiff = 0.5    below this = indifference
*   eps_backfire = 4.0  backfire zone
*   eps_irrel = 6.0     above this = irrelevance    
*   eps_assim = 2.0     assimilation zone

In [15]:
def drf(discrepancy,eps_indiff = 0.5,eps_assim = 2.0,eps_backfire = 4.0 , eps_irrel = 6.0):

    if discrepancy < eps_indiff:
        return 0.0  # indifference
    elif discrepancy < eps_assim:
        return +1.0  # assimilation (pull closer)
    elif discrepancy < eps_backfire:
        return -1.0  # backfire (push away)
    elif discrepancy < eps_irrel:
        return 0.0  # ambivalence (ignored)
    else:
        return 0.0  # irrelevance

In [16]:
def row_normalize(A: np.ndarray, eps=1e-8) -> np.ndarray:
        row_sum = A.sum(axis=1, keepdims=True)
        row_sum = np.clip(row_sum, eps, None)
        return A / row_sum


def adj_to_coo(adj: np.ndarray):
        row, col = np.nonzero(adj)
        edge_index = np.vstack([row, col])
        edge_attr = adj[row, col]
        return edge_index, edge_attr

In [ ]:
# def ensure_non_empty(self, A: np.ndarray) -> np.ndarray:
#       y = A.argmax(axis=1)
#       A_hard = np.zeros_like(A)
#       A_hard[np.arange(len(y)), y] = 1.0
#       counts = np.bincount(y, minlength=self.num_districts)
#       for d in range(self.num_districts):
#           if counts[d] == 0:
#               donor = counts.argmax()
#               donor_nodes = np.where(y == donor)[0]
#               weakest = donor_nodes[np.argmin(A[donor_nodes, donor])]
#               A_hard[weakest, :] = 0.0
#               A_hard[weakest, d] = 1.0
#               y[weakest] = d
#               counts = np.bincount(y, minlength=self.num_districts)
#       return A_hard

## **Population equality**
###(districts must have nearly equal populations)
Compute population deviation score.
    
    population: np.ndarray shape (N,) -> population of each voter/node
    labels: np.ndarray shape (N,) -> district assignment
    num_districts: int
    
    Returns: float (sum of deviations from ideal)


In [17]:
def population_equality(population, labels, num_districts):

    total_pop = np.sum(population)
    ideal = total_pop / num_districts
    deviation = 0.0

    for d in range(num_districts):
        members = np.where(labels == d)[0]

        if len(members) == 0:
            continue

        dist_pop = np.sum(population[members])
        deviation += abs(dist_pop - ideal)
    return deviation


## **check contiguity**
###A district is contiguous if all its members are connected to each other in the graph.
This function Check whether each district forms a connected component.
    
    edge_index: np.ndarray shape (2, E) -> edges
    labels: np.ndarray shape (N,) -> district assignment of each node
    num_districts: int
    
    Returns: bool (True if all districts are contiguous)

In [18]:
from collections import deque

def check_contiguity(edge_index, labels, num_districts):

    N = len(labels)
    adjacency = [[] for _ in range(N)]
    for u, v in edge_index.T:
        adjacency[u].append(v)
        adjacency[v].append(u)

    for d in range(num_districts):
        members = np.where(labels == d)[0]
        if len(members) == 0:
            continue  # empty district handled elsewhere

        # BFS search, using queue
        visited = set()
        queue = deque([members[0]])
        visited.add(members[0])

        while queue:
            node = queue.popleft()
            for nei in adjacency[node]:
                if labels[nei] == d and nei not in visited:
                    visited.add(nei)
                    queue.append(nei)

        if len(visited) != len(members):
            return False  # not contiguous
    return True

## **compactness score**
###  Compactness means districts should not be “spread out” with too many edges crossing between districts.This function computes compactness as ratio of cut edges to total edges.
    
    edge_index (social connection): np.ndarray shape (2, E)
    labels (district labels): np.ndarray shape (N,)
    
    Returns: float (compactness score)

In [19]:
def compactness_score(edge_index, labels):

    total_edges = edge_index.shape[1]
    cut_edges = 0

    for u, v in edge_index.T:
        if labels[u] != labels[v]:
            cut_edges += 1
    return cut_edges / total_edges if total_edges > 0 else 0.0